In [6]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')

## Read data and process data labels

In [7]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [8]:
data=read.csv('Monthly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [9]:
dim(data)

[1] 48000    21

In [10]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [11]:
dlist= load('Monthly_ets_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [12]:
dim(MASE)

[1] 48000     5     4

In [13]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [14]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [15]:
nanum

[1]     4     5     7     8    41    42    43    56    58    59    61    62
   [13]    63    64    65    66    67    68    69    72    73    74    75    78
   [25]    79    81    82    83    84    85    86    87    88    91   103   104
   [37]   106   107   108   109   110   111   112   113   131   132   133   134
   [49]   135   136   137   141   143   147   153   154   155   160   161   162
   [61]   165   167   170   173   191   192   194   195   203   204   205   207
   [73]   215   216   217   218   219   220   221   222   223   226   227   228
   [85]   229   230   231   232   236   238   239   240   241   242   246   247
   [97]   248   252   253   254   255   256   257   258   259   261   262   263
  [109]   264   265   266   271   278   280   282   283   284   286   287   289
  [121]   290   292   293   295   296   297   298   300   303   313   314   315
  [133]   317   318   323   325   326   327   341   344   345   346   347   348
  [145]   349   354   365   366   373   384   388   389   391   396   398   399
  [157]   400   401   402   405   410   411   412   414   420   423   424   426
  [169]   427   434   435   436   439   440   441   447   460   461   462   463
  [181]   469   470   471   472   475   476   477   480   484   503   516   517
  [193]   544   564   566   574   575   576   577   586   591   594   610   612
  [205]   613   634   642   643   645   692   722   737   738   743   745   760
  [217]   765   773   774   777   782   790   800   801   808   843   845   875
  [229]   910   926   953   954   955   958  1064  1087  1112  1224  1232  1271
  [241]  1272  1273  1305  1352  1374  1379  1424  1425  1485  1512  1642  1665
  [253]  1684  1685  1723  1724  1729  1746  1768  1777  1781  1846  1849  1851
  [265]  1860  1861  1862  1913  1914  1949  1950  2040  2077  2130  2136  2139
  [277]  2140  2144  2147  2148  2149  2150  2151  2191  2192  2193  2196  2232
  [289]  2233  2267  2361  2373  2380  2383  2393  2404  2406  2416  2417  2418
  [301]  2419  2421  2462  2508  2509  2511  2582  2586  2587  2588  2589  2590
  [313]  2591  2592  2593  2594  2627  2628  2629  2643  2645  2648  2649  2650
  [325]  2674  2675  2678  2679  2680  2682  2685  2712  2721  2740  2741  2742
  [337]  2743  2744  2745  2746  2747  2748  2749  2750  2751  2752  2753  2754
  [349]  2755  2756  2757  2758  2759  2760  2761  2762  2763  2764  2765  2766
  [361]  2767  2768  2769  2770  2771  2772  2773  2774  2775  2776  2777  2778
  [373]  2779  2780  2781  2782  2783  2784  2785  2786  2787  2788  2789  2790
  [385]  2791  2792  2793  2794  2795  2796  2797  2798  2799  2800  2801  2802
  [397]  2803  2804  2805  2806  2836  2837  2838  2839  2840  2841  2842  2843
  [409]  2844  2845  2846  2847  2848  2849  2850  2851  2866  2867  2900  2901
  [421]  2911  2912  2913  2914  2915  2928  2952  2954  2955  2956  2964  2966
  [433]  2967  2968  2969  2986  2992  3006  3007  3011  3013  3014  3035  3036
  [445]  3037  3047  3050  3052  3053  3054  3069  3071  3072  3073  3074  3075
  [457]  3076  3078  3080  3081  3084  3116  3119  3139  3140  3141  3142  3148
  [469]  3161  3162  3163  3164  3165  3166  3198  3212  3213  3237  3267  3268
  [481]  3269  3270  3284  3329  3380  3385  3387  3390  3392  3466  3467  3468
  [493]  3469  3470  3542  3543  3544  3545  3546  3547  3548  3549  3550  3551
  [505]  3552  3553  3554  3555  3556  3557  3603  3614  3616  3619  3802  3891
  [517]  3903  3910  3911  3912  3913  3914  3915  3925  3936  3940  3948  4080
  [529]  4081  4082  4083  4084  4085  4086  4087  4088  4089  4090  4091  4092
  [541]  4093  4098  4100  4102  4103  4105  4108  4109  4110  4111  4112  4113
  [553]  4114  4115  4116  4117  4118  4120  4121  4122  4123  4124  4125  4126
  [565]  4127  4128  4129  4130  4131  4132  4133  4134  4136  4137  4138  4139
  [577]  4140  4141  4142  4143  4144  4145  4146  4147  4148  4149  4223  4224
  [589]  4225  4226  4230  4371  4427  4428  4429  4430  4431  4432  4433  4434
  [6

In [16]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [17]:
table(realbestmin)

realbestmin
    0     1     2     3     4 
10802  7568  7038 13121  9471 

In [18]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [19]:
realbestmean

4
0
3
3
0
2
0
1
1
4
0


In [20]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [21]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [22]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [23]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [24]:
end_time = Sys.time()

In [25]:
time_matrix[1,]=end_time-start_time

In [26]:
end_time-start_time

Time difference of 13.73029 secs

## Target the interval where the actual error is minimum

In [27]:
start_time = Sys.time()

In [28]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [29]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [30]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.744496 
[2]	train-rmse:1.510492 
[3]	train-rmse:1.376351 
[4]	train-rmse:1.300625 
[5]	train-rmse:1.257659 
[6]	train-rmse:1.231372 
[7]	train-rmse:1.215434 
[8]	train-rmse:1.205718 
[9]	train-rmse:1.196664 
[10]	train-rmse:1.188797 
[11]	train-rmse:1.182641 
[12]	train-rmse:1.177646 
[13]	train-rmse:1.173928 
[14]	train-rmse:1.168451 
[15]	train-rmse:1.164474 
[16]	train-rmse:1.163265 
[17]	train-rmse:1.161488 
[18]	train-rmse:1.158318 
[19]	train-rmse:1.155585 
[20]	train-rmse:1.152543 
[21]	train-rmse:1.150242 
[22]	train-rmse:1.147679 
[23]	train-rmse:1.144694 
[24]	train-rmse:1.140314 
[25]	train-rmse:1.135759 
[26]	train-rmse:1.132270 
[27]	train-rmse:1.130277 
[28]	train-rmse:1.126917 
[29]	train-rmse:1.123256 
[30]	train-rmse:1.121380 
[31]	train-rmse:1.120226 
[32]	train-rmse:1.119672 
[33]	train-rmse:1.116941 
[34]	train-rmse:1.115189 
[35]	train-rmse:1.111656 
[36]	train-rmse:1.109088 
[37]	train-rmse:1.106268 
[38]	train-rmse:1.105336 
[39]	train-rmse:1.101

In [31]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.503259 
[2]	train-mlogloss:1.434239 
[3]	train-mlogloss:1.384629 
[4]	train-mlogloss:1.346363 
[5]	train-mlogloss:1.316929 
[6]	train-mlogloss:1.292653 
[7]	train-mlogloss:1.272093 
[8]	train-mlogloss:1.254391 
[9]	train-mlogloss:1.240038 
[10]	train-mlogloss:1.226610 
[11]	train-mlogloss:1.214607 
[12]	train-mlogloss:1.204220 
[13]	train-mlogloss:1.194095 
[14]	train-mlogloss:1.184751 
[15]	train-mlogloss:1.178457 
[16]	train-mlogloss:1.170384 
[17]	train-mlogloss:1.163027 
[18]	train-mlogloss:1.157270 
[19]	train-mlogloss:1.150285 
[20]	train-mlogloss:1.144537 
[21]	train-mlogloss:1.139488 
[22]	train-mlogloss:1.135976 
[23]	train-mlogloss:1.129830 
[24]	train-mlogloss:1.126857 
[25]	train-mlogloss:1.121331 
[26]	train-mlogloss:1.117183 
[27]	train-mlogloss:1.112353 
[28]	train-mlogloss:1.107409 
[29]	train-mlogloss:1.103046 
[30]	train-mlogloss:1.097870 
[31]	train-mlogloss:1.093839 
[32]	train-mlogloss:1.090632 
[33]	train-mlogloss:1.087945 
[34]	train-mlogloss

In [32]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score 2.064929


In [33]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score -1.492441
[LightGBM] [Info] Start training from score -1.855890
[LightGBM] [Info] Start training from score -1.920403
[LightGBM] [Info] Start training from score -1.295571
[LightGBM] [Info] Start training from score -1.616596


In [34]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [35]:

start_time = Sys.time()

In [36]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [37]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [38]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.532633 
[2]	train-rmse:1.370792 
[3]	train-rmse:1.278874 
[4]	train-rmse:1.226902 
[5]	train-rmse:1.195900 
[6]	train-rmse:1.176730 
[7]	train-rmse:1.164554 
[8]	train-rmse:1.154729 
[9]	train-rmse:1.147997 
[10]	train-rmse:1.140910 
[11]	train-rmse:1.134630 
[12]	train-rmse:1.132201 
[13]	train-rmse:1.126339 
[14]	train-rmse:1.123621 
[15]	train-rmse:1.120317 
[16]	train-rmse:1.117561 
[17]	train-rmse:1.116590 
[18]	train-rmse:1.112066 
[19]	train-rmse:1.107970 
[20]	train-rmse:1.106254 
[21]	train-rmse:1.103890 
[22]	train-rmse:1.101447 
[23]	train-rmse:1.098131 
[24]	train-rmse:1.095638 
[25]	train-rmse:1.092503 
[26]	train-rmse:1.091029 
[27]	train-rmse:1.086652 
[28]	train-rmse:1.081578 
[29]	train-rmse:1.079111 
[30]	train-rmse:1.078069 
[31]	train-rmse:1.075377 
[32]	train-rmse:1.072366 
[33]	train-rmse:1.069112 
[34]	train-rmse:1.068531 
[35]	train-rmse:1.065744 
[36]	train-rmse:1.062124 
[37]	train-rmse:1.058287 
[38]	train-rmse:1.056671 
[39]	train-rmse:1.053

In [39]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.510112 
[2]	train-mlogloss:1.444632 
[3]	train-mlogloss:1.397831 
[4]	train-mlogloss:1.363214 
[5]	train-mlogloss:1.334347 
[6]	train-mlogloss:1.310588 
[7]	train-mlogloss:1.290057 
[8]	train-mlogloss:1.272954 
[9]	train-mlogloss:1.257032 
[10]	train-mlogloss:1.242741 
[11]	train-mlogloss:1.230284 
[12]	train-mlogloss:1.219428 
[13]	train-mlogloss:1.209278 
[14]	train-mlogloss:1.200395 
[15]	train-mlogloss:1.191238 
[16]	train-mlogloss:1.182135 
[17]	train-mlogloss:1.175043 
[18]	train-mlogloss:1.169234 
[19]	train-mlogloss:1.163870 
[20]	train-mlogloss:1.158557 
[21]	train-mlogloss:1.152880 
[22]	train-mlogloss:1.147675 
[23]	train-mlogloss:1.144121 
[24]	train-mlogloss:1.139332 
[25]	train-mlogloss:1.135391 
[26]	train-mlogloss:1.130880 
[27]	train-mlogloss:1.126775 
[28]	train-mlogloss:1.123990 
[29]	train-mlogloss:1.120643 
[30]	train-mlogloss:1.116438 
[31]	train-mlogloss:1.112343 
[32]	train-mlogloss:1.107210 
[33]	train-mlogloss:1.104307 
[34]	train-mlogloss

In [40]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011659 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score 1.664496


In [41]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014407 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score -1.189411
[LightGBM] [Info] Start training from score -1.756492
[LightGBM] [Info] Start training from score -1.748808
[LightGBM] [Info] Start training from score -1.378365
[LightGBM] [Info] Start training from score -2.333266


In [42]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [43]:
end_time-start_time

Time difference of 3.955712 mins

## predict

In [44]:
start_time = Sys.time()

In [45]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [46]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [47]:
xgbregmin

[1]  3.537035465  3.015016317  3.787375450  2.275607824  1.425353527
    [6]  2.767505407  1.542747855  1.960802794  1.172779202  3.190021276
   [11]  0.640338361  1.102308512  2.903605938  0.273516268  2.767649412
   [16]  0.817533195  2.029671669  2.397087574  1.121592164  2.635497808
   [21]  2.233649015  2.103090525  1.876995921  3.838495970  3.525894165
   [26]  3.708957434  2.355682850  2.753593922  3.231353521  3.143670321
   [31]  2.368689775  3.019098997  3.326461554  3.354125500  2.951439619
   [36]  2.963923216  2.368508101  3.549773932  2.894336700  2.857729435
   [41]  1.518637061  1.086492538  1.044500113  2.793335199  0.456784576
   [46]  1.194530249  1.633284092  2.795788050  2.612493038  3.188499451
   [51]  0.590109825  3.150480509  2.958876133  3.372874737  3.747317791
   [56]  1.888737202  4.003897667  0.920087993  1.229411721  3.205281734
   [61]  1.651640296  1.246389389  2.285354614  0.381579846  0.885594964
   [66]  1.244346976  0.434380054  1.495283604  1.496914029  0.938522160
   [71]  0.807112038  0.746917784  1.501572132  0.909395874  0.946343780
   [76]  0.804037154  2.552056074  1.337936759  1.498929739  0.304203540
   [81]  0.329176992  1.109881163  1.106086731  1.343262792  1.638430357
   [86]  1.400079012  1.315061927  0.785825372  2.073958397  1.971091509
   [91]  2.074072599  3.959519386  3.188488960  2.669611216  2.622905731
   [96]  2.274791002  2.679415703  2.548236847  2.536904573  1.110713720
  [101]  3.445671797  2.791774750  1.661565781  1.054845214  3.249874115
  [106]  1.453648329  2.298884153  2.244328737  2.200058222  2.649425745
  [111]  1.636708140  1.890677214  2.315013170  3.810882807  2.309706450
  [116]  3.568832159  3.668184757  3.762693167  3.883213997  3.598820925
  [121]  3.803468466  4.039214134  3.882799864  3.664754152  3.340471983
  [126]  3.235884905  3.261372566  3.261621475  3.270318985  3.375621796
  [131]  1.596012950  0.963921845  2.271381617  2.094428301  1.053096771
  [136]  1.541147590  1.688913107  3.321871042  3.321384430  2.935531139
  [141]  1.590773225  3.055374861  1.570601106  3.366013527  3.263688326
  [146]  2.913594723  1.467957139  3.729769707  4.097700596  3.873649836
  [151]  3.606102467  4.123306274  0.499671429  0.531388223  1.690141916
  [156]  3.468538523  3.234663725  4.083711624  3.371966124  1.933504939
  [161]  2.429549694  1.479542136  3.407205343  3.416502953  0.826274276
  [166]  3.705495119  1.074835658  2.944161415  3.904385090  1.321339607
  [171]  3.386597395  3.521092415  1.883258820  3.098897934  3.375282526
  [176]  3.260673285  2.843869448  2.448348999  2.251554489  3.652767181
  [181]  3.249355555  2.798304319  3.035277367  2.654120922  1.588656664
  [186]  3.572629929  3.279197454  4.132867813  3.641221523  3.673603058
  [191]  1.410192013  1.774134636  3.446583271  0.911748886  1.010382295
  [196]  3.080675602  3.246381521  3.491932154  3.171076059  3.626664639
  [201]  2.686195612  3.290303707  2.575670481  1.544497132  1.119027138
  [206]  3.443021536  1.607543230  2.957745552  3.176691055  4.034152031
  [211]  3.608119011  2.975990057  2.087774277  3.133313179  1.018137336
  [216]  1.113463044  1.427227497  1.212299943  1.460306525  1.208140135
  [221]  1.157938600  1.609641790  1.686793923  2.240583897  2.481278896
  [226]  1.583602309  1.686793923  2.029549837  1.297717333  2.111411810
  [231]  2.029549837  2.390997648  3.303501606  2.905313015  2.145085573
  [236]  2.218467712  3.168719530  1.739507318  1.844249725  2.103023529
  [241]  1.217886090  1.488517523  2.580206871  2.416698933  3.154749870
  [246]  2.396273375  1.298695445  1.030659676  1.894822478  2.739644289
  [251]  3.408190250  1.846933007  2.080567837  2.209426880  1.867416382
  [256]  1.305644870  1.066221714  1.310511589  1.167728543  2.686669350
  [261]  1.409510851  1.749649644  1.269479752  1.054466009  1.931667328
  [266]  2.476366282  2.812339783  3.935842752  3.148342848  3.699413061
  [271]  1.068280697  2.163345098  3.144991398  3.453844547  2

In [48]:
xgbclsmin

[1] 4 4 4 3 0 4 3 3 1 4 0 1 4 0 4 1 2 4 1 4 1 3 4 4 4 4 4 4 3 4 3 4 3 4 3 3
   [37] 4 4 3 4 1 0 0 4 0 0 0 4 4 4 0 3 4 4 4 3 4 3 0 4 0 0 3 0 0 0 0 3 3 0 0 0
   [73] 3 2 0 0 4 0 0 0 0 0 3 3 1 3 3 0 2 4 0 4 3 4 4 2 4 1 4 1 4 4 1 1 4 0 3 3
  [109] 0 3 3 3 3 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 0 3 3 0 1 1 3 4 3 1 4 1 4
  [145] 3 4 0 4 4 4 4 4 0 0 3 4 3 4 4 3 3 0 4 4 0 4 0 3 4 0 4 4 3 3 3 4 3 3 2 4
  [181] 4 4 4 4 1 4 4 4 4 4 1 3 3 0 0 3 3 4 4 4 4 4 3 3 0 4 0 3 3 4 4 4 2 4 0 0
  [217] 0 0 1 0 0 0 3 4 4 3 3 3 2 2 3 3 3 4 0 3 3 3 3 3 1 1 4 0 4 3 0 1 0 4 4 0
  [253] 2 2 0 3 1 0 0 3 2 3 1 0 3 3 4 4 4 4 0 4 4 4 2 4 4 0 4 1 4 3 2 3 4 0 0 4
  [289] 3 3 4 2 3 3 1 2 2 0 4 3 3 0 3 4 4 4 3 3 3 4 3 4 2 1 0 3 3 0 2 0 3 0 0 0
  [325] 0 3 0 4 3 3 4 4 4 4 2 4 4 4 4 0 0 4 4 4 1 3 3 2 0 4 4 4 4 0 3 4 4 4 4 1
  [361] 4 4 1 4 0 0 4 4 4 3 3 4 0 3 4 0 2 3 2 4 4 2 4 0 4 4 4 3 1 4 1 4 3 4 4 2
  [397] 4 3 0 3 3 3 3 2 2 4 1 0 3 0 0 0 2 3 4 4 3 3 1 3 4 3 3 0 2 3 3 4 4 4 4 4
  [433] 4 0 1 2 4 4 0 3 1 0 4 4 4 4 3 3 0 4 3 4 3 4 4 4 3 3 0 2 3 3 3 0 3 4 3 4
  [469] 0 1 3 3 0 3 2 0 1 3 0 3 4 3 2 3 2 3 3 1 4 3 4 3 3 3 4 4 4 0 2 2 0 3 2 2
  [505] 2 3 2 4 4 2 4 2 2 4 4 0 3 3 4 3 4 4 3 4 3 4 0 1 4 3 4 3 2 0 2 4 4 3 2 1
  [541] 4 2 2 2 2 2 4 2 2 4 3 0 4 4 4 1 4 0 1 2 1 3 4 3 2 2 2 2 3 3 1 0 1 1 0 0
  [577] 0 1 2 1 0 3 3 3 1 2 1 4 4 0 1 4 4 2 4 2 4 4 4 4 4 3 1 2 2 4 4 4 4 3 3 3
  [613] 3 4 2 4 1 4 2 4 4 4 3 4 3 4 3 4 4 2 4 3 3 1 4 4 4 4 3 3 3 4 1 0 3 2 3 0
  [649] 2 4 4 3 4 4 3 3 0 4 4 2 1 3 0 4 2 3 4 4 4 3 3 3 3 0 2 4 4 4 4 0 4 0 2 4
  [685] 0 2 0 4 3 1 0 3 4 0 0 4 0 3 2 4 3 4 3 4 4 0 0 2 0 0 0 3 3 0 3 3 0 1 4 4
  [721] 3 2 1 1 0 3 4 1 4 4 3 3 0 4 4 0 2 3 1 4 4 4 3 0 0 0 4 3 3 0 1 4 0 4 0 1
  [757] 4 0 0 2 3 1 1 0 3 4 4 4 4 0 0 3 3 2 4 3 2 0 0 1 1 2 3 0 4 1 4 2 0 2 4 4
  [793] 0 0 4 3 3 3 3 2 0 0 4 3 1 1 1 3 3 4 1 4 3 0 1 0 0 4 3 0 2 4 2 0 3 4 4 4
  [829] 4 4 3 4 0 0 4 1 3 2 4 4 4 0 0 3 3 4 4 4 4 4 4 3 0 4 3 4 3 2 4 3 0 3 3 0
  [865] 4 4 4 0 4 0 0 4 0 4 3 4 4 4 4 0 4 4 4 2 0 4 2 2 1 4 4 0 4 4 0 3 0 3 4 3
  [901] 0 4 4 3 4 4 1 4 0 0 3 4 4 3 0 4 3 0 4 4 4 4 0 4 0 2 3 0 3 3 1 1 1 1 0 3
  [937] 1 4 4 3 3 1 0 4 4 4 1 0 3 0 0 2 0 1 2 0 0 3 3 0 0 3 3 3 3 4 4 3 0 3 0 0
  [973] 4 3 3 0 1 1 4 4 4 4 4 4 4 4 4 4 4 4 1 3 3 0 1 3 2 2 4 0 0 4 3 4 1 0 0 4
 [1009] 3 4 0 0 3 4 0 4 2 4 4 3 0 4 1 2 4 1 1 4 4 3 0 2 1 2 4 2 4 2 2 1 1 0 4 4
 [1045] 2 4 0 1 0 2 0 4 0 0 4 0 4 4 2 4 4 0 3 0 4 4 3 4 0 2 1 0 4 3 4 0 4 1 0 4
 [1081] 3 4 4 0 4 2 1 4 4 3 0 2 4 0 1 1 1 2 0 2 3 1 4 3 4 3 1 4 4 3 3 3 3 4 3 3
 [1117] 3 0 0 0 4 0 3 0 0 3 3 3 3 2 4 0 0 4 0 3 4 0 3 4 4 3 1 3 1 3 3 3 3 4 3 0
 [1153] 0 0 1 0 4 3 0 0 0 0 0 4 4 3 0 4 0 4 4 3 3 3 0 4 3 3 1 4 0 2 4 3 0 4 1 4
 [1189] 4 4 2 1 4 0 3 0 3 3 0 0 1 3 4 1 0 0 0 1 0 4 3 4 0 1 3 2 4 1 1 4 0 3 4 3
 [1225] 2 4 4 3 0 3 0 0 3 2 3 0 4 0 3 0 0 3 4 4 3 3 0 0 4 3 2 3 1 0 4 3 4 3 2 0
 [1261] 4 3 4 0 4 3 4 4 4 3 0 3 3 4 4 2 2 0 4 4 4 3 2 3 1 4 4 0 4 1 2 0 4 4 4 3
 [1297] 3 4 1 4 4 4 3 3 1 4 4 3 3 3 4 4 0 4 4 4 0 4 4 4 0 4 1 2 4 1 1 2 0 4 4 4
 [1333] 4 4 3 4 4 0 4 0 2 3 2 4 4 0 4 3 4 0 4 0 3 4 0 4 4 4 4 4 0 1 0 4 0 4 4 3
 [1369] 1 4 0 3 4 0 3 4 3 4 0 4 4 4 4 4 4 4 2 2 1 4 2 3 4 3 4 4 4 3 0 0 0 0 0 3
 [1405] 3 3 4 4 3 4 4 2 3 3 4 4 2 4 3 0 0 3 0 1 1 4 3 2 1 2 1 0 4 3 4 0 4 0 0 4
 [1441] 4 0 2 2 4 1 0 1 3 3 0 0 3 3 3 1 1 1 3 3 0 0 3 0 4 4 1 3 4 4 0 3 4 4 4 3
 [1477] 3 3 0 3 4 0 4 4 2 0 3 3 0 4 3 1 0 4 4 3 4 3 4 4 4 1 0 4 3 3 3 4 4 3 3 1
 [1513] 3 3 3 3 4 3 3 4 3 2 0 1 3 3 1 0 4 3 4 3 3 3 3 1 1 0 0 3 4 0 3 3 4 4 2 0
 [1549] 2 2 3 4 0 2 4 3 1 0 0 4 0 4 3 4 0 4 0 1 3 0 4 4 0 0 4 3 3 4 0 4 0 4 4 0
 [1585] 0 1 0 3 0 4 2 4 4 0 4 2 2 1 4 4 3 4 2 2 1 4 0 4 4 4 4 2 0 0 4 4 3 2 0 0
 [1621] 1 3 4 4 0 0 2 0 0 4 4 4 0 3 4 0 4 0 4 3 2 3 2 4 1 3 3 3 1 0 4 0 3 0 0 4
 [1657] 4 3 4 3 4 3 4 4 1 3 3 4 4 4 4 3 0 0 4 4 3 4 4 4 4 4 0 0 0 2 4 2 0 1 3 0
 [1693] 4 4 3 0 3 0 0 3 3 4 0 4 4 4 3 3 0 3 3 1 0 4 4 4 0 0 0 0 4 4 3 3 3 4 4 3
 [1729] 1 3 3 4 4 3 3 3 3 3 3 3 4 0 4 4 4 3 3 4 4 4 4 4 0 4 4 3 4 4 4 3 0 3 0 3
 [1765] 4 4 4 0 4 4 1 4 0 3 3 3 2 0 3 0 0 4 4 2 1 3 3 4 0 4 0 4 1 1 4 3 0 2 1 0
 [18

In [49]:
lgbregmin

[1] 3.0780490 2.6442791 3.4486846 2.1652199 1.8773389 2.4905587 1.9758937
    [8] 1.9680984 2.0248550 2.3728698 1.4440479 1.7610651 2.5547651 1.1325162
   [15] 2.8563025 1.4416977 1.7803647 1.6304493 1.8851204 2.2588153 1.9631539
   [22] 1.9991529 2.2552693 3.5325671 3.4829914 3.5878169 3.4236608 2.7817987
   [29] 3.0175557 3.3009986 2.4537077 3.3034572 3.1105143 2.9600706 2.8714505
   [36] 3.0110051 2.3003062 3.3481387 2.9574654 2.7675451 1.3578640 1.3951693
   [43] 1.2935818 2.7277212 1.9236382 1.8075924 2.1054493 2.3477147 2.0550239
   [50] 2.4090226 1.1131828 3.5042500 2.5406227 2.7349739 3.6096331 1.9169256
   [57] 2.1501222 1.0587643 1.2225667 2.2258037 1.7225589 2.0699046 2.3964247
   [64] 1.4794012 1.0358453 1.6181167 1.0260511 1.4816378 1.8061403 1.3093361
   [71] 1.5193798 0.9760501 1.7032568 1.3770944 1.4222255 2.5145780 2.8827033
   [78] 1.1329176 1.5286035 1.4317829 0.8406056 1.4566486 0.9611754 1.7401275
   [85] 1.5855236 1.8851239 1.6886347 1.2640040 1.8375741 2.0969850 1.8671988
   [92] 3.5082522 3.5245510 2.7424994 2.5361870 2.4815177 2.5454432 2.5953603
   [99] 2.6533999 1.4605004 2.8690799 2.6637062 1.5699575 1.2229453 3.0973364
  [106] 1.5132516 2.4129059 2.0862542 1.4305941 2.2070660 1.4732931 1.8885422
  [113] 2.5785292 3.8851470 2.2022975 3.1580751 3.4334032 3.5371295 3.5640047
  [120] 3.8344953 3.5113921 3.6069863 3.8522319 3.4167986 2.9028838 3.0523688
  [127] 3.0514565 3.0375896 3.1983237 3.1244776 1.5215281 1.3531498 1.7215529
  [134] 1.9836948 1.1436061 1.6121330 1.6012427 3.4097206 3.4827553 3.1915294
  [141] 1.4371578 3.2311359 1.4891640 3.2417692 3.3944960 2.3782428 1.4108685
  [148] 3.6348648 3.7350430 3.6210606 3.5458207 3.6719684 0.7210567 0.7068710
  [155] 1.8494108 3.4873415 3.2231627 3.8124474 3.3095800 1.6232895 1.8430222
  [162] 1.4221637 2.8496781 2.7643626 1.3160220 3.5423713 1.1082135 3.2599631
  [169] 3.5433849 1.3487223 3.0174994 3.4956588 1.7799977 3.2808127 3.2316541
  [176] 3.1009493 3.1745015 2.9523397 2.3269849 3.7719696 3.4781183 2.8105061
  [183] 3.0790053 2.8463851 1.9628200 3.6437717 3.4943318 3.5901533 3.1892150
  [190] 3.6144638 1.5157571 1.4249301 3.6235152 1.1447903 1.3491673 3.5553483
  [197] 3.2647648 3.3424536 3.1776154 3.4859676 2.5115749 3.3506914 2.5495014
  [204] 2.0178963 1.5036579 3.5491524 1.4729825 3.3147307 3.1952672 3.5918620
  [211] 3.6731133 3.3382204 2.5807191 2.9553996 1.5130713 1.1523319 1.2421329
  [218] 1.1101393 1.2018142 1.0553539 0.9232026 1.2683517 1.6029970 2.4036188
  [225] 2.4906838 1.5078144 1.6029970 1.9018279 1.4442996 2.0158973 1.9018279
  [232] 2.3103510 3.0969540 3.2013279 1.8436882 1.9522612 3.1459303 1.6687646
  [239] 1.7584594 1.9976985 1.3510464 1.7999571 2.5177468 2.2741923 3.2436175
  [246] 1.4385506 1.1900361 1.1603406 2.0984040 3.1779534 3.2582740 1.5154182
  [253] 2.2278210 2.1283082 1.2058407 1.3599447 1.5257776 1.2914974 1.2594930
  [260] 2.8166877 1.4388509 1.6695084 1.1598008 1.1636790 2.0226142 2.6522022
  [267] 3.1684805 3.4969469 3.0346095 3.1984977 1.4461651 2.6527702 3.2014141
  [274] 3.0564411 2.6031293 3.4539030 2.3753639 1.0729664 2.6518769 1.4603663
  [281] 2.3715669 1.4557841 0.9015885 1.5610648 2.9524741 1.1738253 1.1672071
  [288] 2.9232452 1.6296282 1.6680225 2.3746314 1.6405774 1.9562808 1.8411327
  [295] 0.8805556 1.1844139 0.8774794 1.3849279 2.4027893 2.2037914 2.3119775
  [302] 1.7169716 1.8609368 3.3091594 3.2856635 3.0132528 3.4972974 3.5223894
  [309] 3.5434069 2.9841767 2.2455718 2.2938020 1.8307009 1.4827101 1.6202996
  [316] 3.1971214 1.6747252 1.6525493 2.9724428 3.0721449 3.2655717 1.9210177
  [323] 1.0448917 1.8139818 1.7863664 2.2536706 1.1610425 3.3882182 3.5006441
  [330] 2.3727521 2.4537718 3.3371934 3.4610996 3.6679092 3.1354942 3.8696588
  [337] 2.2330885 2.0264826 3.7142889 3.0077134 1.2698656 3.8049826 3.7298913
  [344] 2.7359642 1.2902721 1.4613827 2.6803188 1.5705108 1.3994118 2.7849691
  [351] 3.6253322 3.7678686 3.6456922 1.2484065 3.1013053 3.7406448 2.2065348
  [358] 2.1598215 2.835704

In [50]:
lgbclsmin

0.165306745,0.04381587,0.04112481,0.18752425,5.622283e-01
0.167871159,0.04414212,0.03685897,0.17819604,5.729317e-01
0.034870279,0.02809031,0.08629242,0.24296140,6.077856e-01
0.191681161,0.08011464,0.08343030,0.64472598,4.791467e-05
0.286325723,0.15151876,0.14422734,0.41781827,1.099024e-04
0.203990011,0.13016838,0.10258682,0.11536974,4.478851e-01
0.148693464,0.22948217,0.17285160,0.44886786,1.049113e-04
0.104092522,0.20314868,0.21419308,0.47838643,1.792907e-04
0.267162255,0.30943173,0.07311265,0.11299015,2.373032e-01
0.282806522,0.02108538,0.02386969,0.07675617,5.954822e-01
0.539717776,0.13698788,0.05690056,0.05754443,2.088494e-01


In [51]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [52]:
lgbclsminr

4
4
0
2
0
2
4
1
4
0
0


In [53]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [54]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [55]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [56]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.537035,4,3.078049
4,3.015016,4,2.644279
4,3.787375,0,3.448685
3,2.275608,2,2.165220
0,1.425354,0,1.877339
4,2.767505,2,2.490559


In [57]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [58]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
4,2.395146,4,2.354662
0,1.556739,4,1.846232
3,2.707116,3,3.174982
3,2.102187,2,1.489704
0,1.118452,1,1.265898
2,1.698595,2,1.835499


In [59]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
4,2.395146,4,2.354662
0,1.556739,4,1.846232
3,2.707116,3,3.174982
3,2.102187,2,1.489704
0,1.118452,1,1.265898
2,1.698595,2,1.835499


In [60]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [61]:
result_list=list(preallmin,preallmean,time_matrix)

In [62]:
save(result_list, file = "Monthly_ets_opt_pre_result.RData")

In [63]:
time_matrix

user_time,system_time,elapsed_time
13.730287,13.730287,13.730287
2.096178,2.096178,2.096178
3.955712,3.955712,3.955712
5.861260,5.861260,5.861260


## importance

In [64]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)


              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.24958835 0.115685477 0.06593244
 2:         linearity 0.12039521 0.084360613 0.08405768
 3:         curvature 0.11251232 0.096948217 0.08894706
 4:         seas_acf1 0.07622235 0.086958805 0.06583366
 5: seasonal_strength 0.04323521 0.056110648 0.06440142
 6:           entropy 0.03970185 0.055725913 0.05916634
 7:           x_acf10 0.03681984 0.053632337 0.04721454
 8:       diff1_acf10 0.03495581 0.050056682 0.05610431
 9:            x_acf1 0.03466801 0.043129907 0.04642434
10:             spike 0.03407073 0.066115114 0.05338799
11:            e_acf1 0.03379281 0.049245740 0.05294350
12:           e_acf10 0.03356375 0.044151316 0.05738838
13:       diff2_acf10 0.03180461 0.039757104 0.05541288
14:        diff1_acf1 0.02983267 0.049299253 0.04736270
15:             trend 0.02951974 0.043908098 0.04988147
16:        diff2_acf1 0.02933734 0.044729266 0.0

In [65]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_reg)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.28111525 0.093873372 0.05607667
 2:         linearity 0.15336961 0.086140477 0.09726754
 3:         curvature 0.13620972 0.079521238 0.10154976
 4:         seas_acf1 0.04742035 0.063784187 0.05240620
 5:           entropy 0.04285437 0.106590194 0.06219413
 6: seasonal_strength 0.04241014 0.081244603 0.06749592
 7:             spike 0.03365240 0.063647101 0.05770799
 8:           x_acf10 0.03124760 0.058546511 0.03894780
 9:            e_acf1 0.03028242 0.045555617 0.05424144
10:             trend 0.02927152 0.045459474 0.09400489
11:       diff1_acf10 0.02842867 0.051867535 0.04669657
12:       diff2_acf10 0.02793527 0.046856082 0.05097879
13:            x_acf1 0.02564644 0.047048663 0.04200653
14:           e_acf10 0.02508958 0.040987288 0.05077488
15:        diff1_acf1 0.02271269 0.035917184 0.04506525
16:        diff2_acf1 0.02043536 0.038046331 0.0

In [66]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.16805106 0.092178637 0.06039822
 2:         linearity 0.14492114 0.096969963 0.08000205
 3:         curvature 0.12998942 0.088176557 0.08747505
 4:         seas_acf1 0.06329819 0.084851923 0.06654041
 5: seasonal_strength 0.05518719 0.067674153 0.07304090
 6:           entropy 0.04390918 0.060431051 0.05850438
 7:            e_acf1 0.04247818 0.043189228 0.05625224
 8:             spike 0.04173839 0.057889832 0.05635461
 9:       diff1_acf10 0.03844264 0.055193419 0.05860675
10:       diff2_acf10 0.03718742 0.053288432 0.05783897
11:            x_acf1 0.03657665 0.048832299 0.04775554
12:           x_acf10 0.03544370 0.042855120 0.04074321
13:           e_acf10 0.03443864 0.051064906 0.05405129
14:        diff2_acf1 0.03320987 0.036170567 0.05154323
15:             trend 0.03266840 0.049756312 0.04954701
16:        diff1_acf1 0.03221020 0.055056235 0.0

In [67]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_min_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain      Cover  Frequency
               <char>      <num>      <num>      <num>
 1:            length 0.30637955 0.20137518 0.09806667
 2:         linearity 0.12474458 0.08761270 0.08966667
 3:         curvature 0.12078981 0.13040157 0.09733333
 4:         seas_acf1 0.07599703 0.09271879 0.06853333
 5: seasonal_strength 0.04064109 0.04532068 0.06666667
 6:           entropy 0.03549567 0.04603732 0.05666667
 7:           x_acf10 0.03128584 0.04811995 0.04413333
 8:            x_acf1 0.03027900 0.03320004 0.04393333
 9:       diff1_acf10 0.02980987 0.03737706 0.05500000
10:             spike 0.02791002 0.04294262 0.05000000
11:            e_acf1 0.02697622 0.03664612 0.04686667
12:           e_acf10 0.02607693 0.02608271 0.05060000
13:             trend 0.02592452 0.03764967 0.04326667
14:       diff2_acf10 0.02586930 0.03958787 0.05166667
15:        diff1_acf1 0.02366409 0.03453612 0.04180000
16:        diff2_acf1 0.02173541 0.02832023 0.04393333
17:       

In [68]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain      Cover  Frequency
               <char>      <num>      <num>      <num>
 1:            length 0.21834094 0.19146515 0.09953333
 2:         linearity 0.15458575 0.11635167 0.08313333
 3:         curvature 0.14207721 0.11866744 0.09493333
 4:         seas_acf1 0.06222350 0.08967359 0.06973333
 5: seasonal_strength 0.04809148 0.05461728 0.07133333
 6:           entropy 0.03677896 0.04064846 0.05033333
 7:            e_acf1 0.03595074 0.03731025 0.05180000
 8:             spike 0.03402326 0.04007640 0.05053333
 9:       diff1_acf10 0.03338783 0.03561266 0.05393333
10:           x_acf10 0.03275079 0.03903479 0.04033333
11:             trend 0.03101548 0.03005684 0.04533333
12:       diff2_acf10 0.03051644 0.04589528 0.05340000
13:            x_acf1 0.03038930 0.03767295 0.04280000
14:           e_acf10 0.02903669 0.03022563 0.05046667
15:        diff2_acf1 0.02551023 0.02251366 0.04406667
16:        diff1_acf1 0.02407035 0.03962905 0.04120000
17:       

In [69]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain      Cover  Frequency
               <char>       <num>      <num>      <num>
 1:         linearity 0.287448465 0.14073197 0.10100000
 2:         curvature 0.194069214 0.11044127 0.11866667
 3:            length 0.191778219 0.21431445 0.11500000
 4:         seas_acf1 0.061832258 0.09207881 0.07400000
 5: seasonal_strength 0.034624857 0.07018902 0.07366667
 6:           x_acf10 0.031218873 0.04500022 0.04666667
 7:           entropy 0.024863444 0.05721876 0.04800000
 8:             spike 0.023182662 0.04871022 0.04900000
 9:           e_acf10 0.020635309 0.02111406 0.05133333
10:             trend 0.019834812 0.02533051 0.04266667
11:            e_acf1 0.019668739 0.02716049 0.04300000
12:            x_acf1 0.018701534 0.01974976 0.03433333
13:       diff1_acf10 0.015954165 0.02246837 0.04533333
14:       diff2_acf10 0.015074365 0.03111057 0.04033333
15:            trough 0.011617871 0.01018425 0.03000000
16:        diff2_acf1 0.011136820 0.02030582 0.0

In [70]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

In [71]:
importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

In [72]:
importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

In [73]:
importance=cbind(importance_matrix1[,1],importance_matrix_mean)

In [74]:
write.csv(importance,'M_ets_imp_mean.csv')

In [75]:
importance_matrix2[order(importance_matrix2$Feature),]

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.16207740,0.078317788,0.09962144
diff1_acf1,0.02158579,0.045906156,0.04104403
diff1_acf10,0.02882507,0.069616557,0.05160391
diff2_acf1,0.02405325,0.043499693,0.04223949
diff2_acf10,0.02950328,0.041035959,0.04841602
e_acf1,0.03127400,0.055928367,0.05698346
e_acf10,0.03195448,0.037839986,0.05439331
entropy,0.03368065,0.089438407,0.05319785
length,0.15054097,0.090157031,0.05200239


In [77]:
# 使用saveRDS保存模型到.rds文件
saveRDS(xgb_mean_reg, file = "M_etsxgbreg.rds")
saveRDS(xgb_mean_cl, file = "M_etsxgbcls.rds")
saveRDS(lgb_mean_reg, file = "M_etslgbreg.rds")
saveRDS(lgb_mean_cl, file = "M_etslgbcls.rds")